In [1]:
LIGHT_IMPORT = True

%run -i "Calc_Embeddings_Model Training.ipynb"

# quick sanity checks
print("df shape:", df.shape if 'df' in globals() else 'df missing')
print("have get_matches?", 'get_matches' in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [7]:
df.columns.tolist()

['user_id',
 'age',
 'status',
 'sex',
 'orientation',
 'body_type',
 'diet',
 'drinks',
 'drugs',
 'education',
 'ethnicity',
 'height',
 'income',
 'job',
 'last_online',
 'location',
 'offspring',
 'pets',
 'religion',
 'sign',
 'smokes',
 'speaks',
 'essay0',
 'essay1',
 'essay2',
 'essay3',
 'essay4',
 'essay5',
 'essay6',
 'essay7',
 'essay8',
 'essay9',
 'bio_text',
 'lat',
 'lon',
 'diet_c',
 'drinks_c',
 'smokes_c',
 'drugs_c']

In [3]:
df.head(5)

,user_id,age,status,sex,orientation,body_type,diet,drinks,drugs,education,...,essay7,essay8,essay9,bio_text,lat,lon,diet_c,drinks_c,smokes_c,drugs_c
0,0,22,single,m,straight,a little extra,strictly anything,socially,never,working on college/university,...,trying to find someone to hang out with. i am ...,i am new to california and looking for someone...,you want to be swept off your feet! you are ti...,about me: i would love to think that i was so...,37.653540,-122.416866,omnivore,light,light,none
1,1,35,single,m,straight,average,mostly other,often,sometimes,working on space camp,...,,i am very open and will share just about anyth...,,i am a chef: this is what that means. 1. i am ...,37.804456,-122.271356,other,heavy,none,sometimes
2,2,38,available,m,straight,thin,anything,socially,NaN,graduated from masters program,...,viewing. listening. dancing. talking. drinking...,"when i was five years old, i was known as ""the...","you are bright, open, intense, silly, ironic, ...","i'm not ashamed of much, but writing public te...",37.779259,-122.419329,omnivore,light,none,
3,3,23,single,m,straight,thin,vegetarian,socially,NaN,working on college/university,...,,,you feel so inclined.,i work in a library and go to school. . . <e> ...,37.870839,-122.272863,vegetarian,light,none,
4,4,29,single,m,straight,athletic,NaN,socially,never,graduated from college/university,...,,,,hey how's it going? currently vague on the pro...,37.779259,-122.419329,,light,none,none


In [14]:
import pandas as pd

# Take only the first 10 characters (YYYY-MM-DD)
df["last_online_clean"] = df["last_online"].astype(str).str.slice(0, 10)

# Parse as date
df["last_online_dt"] = pd.to_datetime(df["last_online_clean"], errors="coerce", utc=True)

# Now build recency features
df["recency_rank"] = df["last_online_dt"].rank(ascending=True)        # oldest=1
mask = df["last_online_dt"].notna()

# Normalized 0–1 scale within dataset
min_dt, max_dt = df["last_online_dt"].min(), df["last_online_dt"].max()
if pd.notna(min_dt) and pd.notna(max_dt) and max_dt > min_dt:
    span = (max_dt - min_dt).total_seconds()
    df["recency_norm"] = ((df["last_online_dt"] - min_dt).dt.total_seconds() / span).fillna(0.0)
else:
    df["recency_norm"] = 0.0

print(df[["last_online_dt","recency_norm"]].head(10))

             last_online_dt  recency_norm
0 2012-06-28 00:00:00+00:00      0.991892
1 2012-06-29 00:00:00+00:00      0.994595
2 2012-06-27 00:00:00+00:00      0.989189
3 2012-06-28 00:00:00+00:00      0.991892
4 2012-06-27 00:00:00+00:00      0.989189
5 2012-06-29 00:00:00+00:00      0.994595
6 2012-06-25 00:00:00+00:00      0.983784
7 2012-06-29 00:00:00+00:00      0.994595
8 2012-06-29 00:00:00+00:00      0.994595
9 2012-06-28 00:00:00+00:00      0.991892


In [25]:
def _ensure_text_sim(cands: pd.DataFrame) -> pd.DataFrame:
    c = cands.copy()
    if "text_sim" not in c.columns and "score" in c.columns:
        c["text_sim"] = c["score"]
    c["text_sim"] = c["text_sim"].fillna(0).clip(lower=0)
    return c

In [179]:
# Merge recency_norm, blend, and nudge active users

def apply_weights_tinder(cands,w_text=0.60, w_age=0.10, w_life=0.10, w_loc=0.10, w_rec=0.10):
    """
    tinder_score = w_text*text_sim + w_age*age_score + w_life*lifestyle_score + w_loc*loc_score + w_rec*recency_norm
    (weights are normalized inside)
    """
    c = cands.copy()
    # ensure required cols exist
    for col in ["text_sim","age_score","lifestyle_score","loc_score"]:
        if col not in c.columns:
            c[col] = 0.0

    # bring in recency_norm from the master df
    c = c.merge(df[["user_id","recency_norm"]], on="user_id", how="left")
    c["recency_norm"] = c["recency_norm"].fillna(0.0).clip(0,1)

    # normalize weights
    total = float(w_text + w_age + w_life + w_loc + w_rec)
    w_text, w_age, w_life, w_loc, w_rec = [w/total for w in (w_text, w_age, w_life, w_loc, w_rec)]

    c["tinder_score"] = (w_text*c["text_sim"] + w_age *c["age_score"] + w_life*c["lifestyle_score"] +
                        w_loc *c["loc_score"] + w_rec *c["recency_norm"])
    return c.sort_values("tinder_score", ascending=False)


In [180]:
def tinder_score(user_id, k=20, pool_k=500, max_km=50,drinks_strict=False, smokes_strict=False, 
                 drugs_strict=False,allow_missing=True,weights=None):
    """
    Tinder-style scorer:
      - Hard filters: orientation, location (<= max_km), age
      - Soft features: text_sim (from semantic score), lifestyle_score, age_score, loc_score
      - Weighted blend -> final_score
    """
    if weights is None:
        weights = {"text":0.70, "age":0.10, "life":0.15, "loc":0.05}

    # 1) Candidate pool (no hard lifestyle gates)
    cands = get_matches(user_id=user_id, k=pool_k,use_orientation=True, use_location=True, max_km=max_km,
        use_age=True,use_diet=False, use_drinks=False, use_smokes=False, use_drugs=False,
        drinks_strict=drinks_strict, smokes_strict=smokes_strict, drugs_strict=drugs_strict,
        allow_missing=True).copy()

    # 2) Component scores
    cands = compute_component_scores(user_id=user_id, cands=cands, max_km=max_km,
        drinks_strict=drinks_strict, smokes_strict=smokes_strict, drugs_strict=drugs_strict,
        allow_missing=allow_missing)
    cands = _ensure_text_sim(cands)  # maps 'score' -> 'text_sim' if present

    # 3) Blend
    ranked = apply_weights_tinder(cands, w_text=weights["text"], w_age=weights["age"],w_life=weights["life"], 
                           w_loc=weights["loc"]).copy()

    # 4) Final shape
    want = ["user_id","tinder_score","text_sim","age_score","lifestyle_score","loc_score","distance_km",
            "location","age","sex","orientation"]
    for col in want:
        if col not in ranked.columns:
            ranked[col] = np.nan
    ranked_top = ranked[want].head(k).copy()
    ranked_top["source"] = "ranked"

    # align random suggestions to same columns
    for col in want:
        if col not in random_potential_matches.columns:
            random_potential_matches[col] = np.nan
    return ranked[want].head(k).reset_index(drop=True)

In [181]:
tinder = tinder_score(user_id=14, k=10, pool_k=500, max_km=50)
tinder.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225


,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,38159,0.836038,0.742732,1.0,1.0,1.0,26.175450,"san carlos, california",32,m,straight
1,3717,0.830340,0.734164,1.0,1.0,1.0,31.203141,"palo alto, california",37,m,straight
2,15046,0.827298,0.728997,1.0,1.0,1.0,23.920071,"san francisco, california",36,m,straight
3,16398,0.825377,0.725979,1.0,1.0,1.0,31.203141,"palo alto, california",35,m,straight
4,35672,0.825033,0.725438,1.0,1.0,1.0,13.449287,"oakland, california",25,m,straight
5,10444,0.823391,0.722858,1.0,1.0,1.0,19.197039,"berkeley, california",22,m,straight
6,47397,0.823200,0.722558,1.0,1.0,1.0,23.920071,"san francisco, california",32,m,straight
7,31015,0.823117,0.723199,1.0,1.0,1.0,23.920071,"san francisco, california",31,m,straight
8,41489,0.822003,0.720677,1.0,1.0,1.0,31.716076,"belvedere tiburon, california",23,m,straight
9,17586,0.820568,0.718421,1.0,1.0,1.0,23.920071,"san francisco, california",24,m,straight


In [121]:
#testing to make sure compute_component_scores works
scored = compute_component_scores(user_id=u, cands=cands, max_km=20,
    drinks_strict=False, smokes_strict=False, drugs_strict=False, allow_missing=True)
scored = _ensure_text_sim(scored)
scored.head(15)

,user_id,age,sex,orientation,location,text_sim,diet_c,drinks_c,smokes_c,drugs_c,lat,lon,age_score,lifestyle_score,loc_score,distance_km,dist_penalty
2,38159,32,m,straight,"san carlos, california",0.742732,,light,none,none,37.504936,-122.261823,1.0,1.00,0.691228,26.175450,6.175450
7,40022,25,m,straight,"san francisco, california",0.736307,omnivore,heavy,light,,37.779259,-122.419329,1.0,1.00,0.803996,23.920071,3.920071
9,25427,29,m,straight,"mountain view, california",0.735156,vegetarian,light,none,none,37.389389,-122.083210,1.0,0.75,0.107034,37.859321,17.859321
10,3717,37,m,straight,"palo alto, california",0.734164,,,light,,37.444329,-122.159847,1.0,1.00,0.439843,31.203141,11.203141
11,28563,29,m,straight,"san francisco, california",0.732910,,light,none,none,37.779259,-122.419329,1.0,1.00,0.803996,23.920071,3.920071
12,39450,25,m,straight,"san francisco, california",0.732575,omnivore,,none,none,37.779259,-122.419329,1.0,1.00,0.803996,23.920071,3.920071
16,23663,27,m,straight,"san mateo, california",0.730931,omnivore,light,none,none,37.496904,-122.333057,1.0,1.00,0.511777,29.764458,9.764458
20,15046,36,m,straight,"san francisco, california",0.728997,omnivore,light,none,none,37.779259,-122.419329,1.0,1.00,0.803996,23.920071,3.920071
21,33656,26,m,straight,"san francisco, california",0.728639,omnivore,light,none,none,37.779259,-122.419329,1.0,1.00,0.803996,23.920071,3.920071
26,46695,31,m,straight,"palo alto, california",0.726020,vegetarian,light,none,none,37.444329,-122.159847,1.0,0.75,0.439843,31.203141,11.203141


In [73]:
#testing to make sure apply_weights_tinder works
ranked = apply_weights_tinder(scored, w_text=0.55, w_age=0.10, w_life=0.10, w_loc=0.10, w_rec=0.15)
ranked[["user_id","tinder_score","text_sim","age_score","lifestyle_score","loc_score","recency_norm","distance_km"]].head(10)

,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,recency_norm,distance_km
8,44908,0.799900,0.636920,1.0,1.0,1.000000,0.997297,13.980956
11,4120,0.797826,0.633149,1.0,1.0,1.000000,0.997297,13.980956
16,57214,0.794873,0.627041,1.0,1.0,1.000000,1.000000,13.980956
27,56064,0.791719,0.621308,1.0,1.0,1.000000,1.000000,13.980956
2,47209,0.790329,0.644579,1.0,1.0,1.000000,0.905405,13.980956
26,7169,0.790105,0.621321,1.0,1.0,1.000000,0.989189,18.918516
15,26536,0.789406,0.627873,1.0,1.0,0.944814,0.997297,21.103721
19,30403,0.788710,0.624682,1.0,1.0,1.000000,0.967568,13.980956
1,43841,0.787977,0.654167,1.0,1.0,0.785908,0.997297,24.281842
35,53269,0.787719,0.615508,1.0,1.0,1.000000,0.994595,13.980956


In [125]:
def tinder_suggestions(user_id, random_sample=10, pool_k=500, max_km=50,drinks_strict=False, 
                       smokes_strict=False, drugs_strict=False,allow_missing=True, weights=None):
    """
    Returns a small set of 'suggested' candidates sampled from the same Tinder pool,
    scored the same way as ranked, and weighted with the same weights.
    - Does NOT return ranked; call tinder_score(...) separately.
    - Use exclude_ids to avoid duplicates with ranked results.
    """
    if weights is None:
        # same defaults you used for tinder_score
        weights = {"text": 0.70, "age": 0.10, "life": 0.15, "loc": 0.05}

    # 1) Pool with only hard gates (orientation, age, location)
    cands = get_matches(user_id=user_id, k=pool_k,use_orientation=True, use_location=True, max_km=max_km,
        use_age=True,use_diet=False, use_drinks=False, use_smokes=False, use_drugs=False,
        drinks_strict=drinks_strict, smokes_strict=smokes_strict, drugs_strict=drugs_strict,
        allow_missing=allow_missing).copy()

    # 2) Component scores (same as ranked path)
    cands = compute_component_scores(user_id=user_id, cands=cands, max_km=max_km,drinks_strict=drinks_strict, 
        smokes_strict=smokes_strict, drugs_strict=drugs_strict,allow_missing=allow_missing)
    cands = _ensure_text_sim(cands)

    # 3) Sample a few suggestions from the *scored* pool
    WANT_COLS = ["user_id","text_sim","age_score","lifestyle_score","loc_score","distance_km","location",
                 "age","sex","orientation"]
    if not set(WANT_COLS).issubset(cands.columns):
        for col in WANT_COLS:
            if col not in cands.columns:
                cands[col] = np.nan

    sugg = cands.sample(n=min(random_sample, len(cands)), random_state=42)[WANT_COLS].copy()

    # 4) Weight them exactly like ranked
    def _blend(df_in):
        return apply_weights_tinder(df_in,w_text=weights.get("text", 0.65),w_age =weights.get("age",  0.10),
                w_life=weights.get("life", 0.15),w_loc =weights.get("loc",  0.05),w_rec =weights.get("rec",  0.05))

    sugg_ranked = _blend(sugg).copy()
    sugg_ranked["source"] = "suggested"

    WANT_COLS_PLUS = ["user_id","tinder_score","text_sim","age_score","lifestyle_score",
                      "loc_score","distance_km","location","age","sex","orientation","source"]
    for col in WANT_COLS_PLUS:
        if col not in sugg_ranked.columns:
            sugg_ranked[col] = np.nan

    return sugg_ranked[WANT_COLS_PLUS].reset_index(drop=True)

In [130]:
# ranked only
ranked = tinder_score(user_id=u, k=20, pool_k=500, max_km=50)
ranked_ids = ranked["user_id"].tolist()
ranked.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225


,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,38159,0.836038,0.742732,1.0,1.0,1.0,26.175450,"san carlos, california",32,m,straight
1,3717,0.830340,0.734164,1.0,1.0,1.0,31.203141,"palo alto, california",37,m,straight
2,15046,0.827298,0.728997,1.0,1.0,1.0,23.920071,"san francisco, california",36,m,straight
3,16398,0.825377,0.725979,1.0,1.0,1.0,31.203141,"palo alto, california",35,m,straight
4,35672,0.825033,0.725438,1.0,1.0,1.0,13.449287,"oakland, california",25,m,straight
5,10444,0.823391,0.722858,1.0,1.0,1.0,19.197039,"berkeley, california",22,m,straight
6,47397,0.823200,0.722558,1.0,1.0,1.0,23.920071,"san francisco, california",32,m,straight
7,31015,0.823117,0.723199,1.0,1.0,1.0,23.920071,"san francisco, california",31,m,straight
8,41489,0.822003,0.720677,1.0,1.0,1.0,31.716076,"belvedere tiburon, california",23,m,straight
9,17586,0.820568,0.718421,1.0,1.0,1.0,23.920071,"san francisco, california",24,m,straight


In [129]:
# suggestions
suggested = tinder_suggestions(user_id=u, random_sample=20, pool_k=500, max_km=50,
                               weights={"text":0.70,"age":0.10,"life":0.15,"loc":0.05})
suggested.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤50 km): 2457
After location (≤50 km): 2457
After age rule: 2225


,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,source
0,10833,0.798922,0.699155,1.0,1.00,1.0,30.700929,"el sobrante, california",33,m,straight,suggested
1,54851,0.797583,0.698112,1.0,1.00,1.0,23.920071,"san francisco, california",27,m,straight,suggested
2,48244,0.796747,0.695700,1.0,1.00,1.0,19.197039,"berkeley, california",23,m,straight,suggested
3,11226,0.792767,0.689729,1.0,1.00,1.0,23.920071,"san francisco, california",31,m,straight,suggested
4,53171,0.789856,0.685749,1.0,1.00,1.0,28.113170,"daly city, california",26,m,straight,suggested
5,8379,0.787841,0.682148,1.0,1.00,1.0,16.481152,"emeryville, california",36,m,straight,suggested
6,46695,0.781504,0.726020,1.0,0.75,1.0,31.203141,"palo alto, california",31,m,straight,suggested
7,57727,0.774239,0.661552,1.0,1.00,1.0,29.764458,"san mateo, california",22,m,straight,suggested
8,13060,0.773779,0.661054,1.0,1.00,1.0,23.920071,"san francisco, california",40,m,straight,suggested
9,14484,0.773126,0.659883,1.0,1.00,1.0,23.920071,"san francisco, california",26,m,straight,suggested


In [131]:
tinder_suggestions_df = pd.concat([ranked.head(k).assign(source="ranked"), suggested],
    ignore_index=True).drop_duplicates("user_id", keep="first")

tinder_suggestions_df.head(15)

,user_id,tinder_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,source
0,38159,0.836038,0.742732,1.0,1.0,1.0,26.175450,"san carlos, california",32,m,straight,ranked
1,3717,0.830340,0.734164,1.0,1.0,1.0,31.203141,"palo alto, california",37,m,straight,ranked
2,15046,0.827298,0.728997,1.0,1.0,1.0,23.920071,"san francisco, california",36,m,straight,ranked
3,16398,0.825377,0.725979,1.0,1.0,1.0,31.203141,"palo alto, california",35,m,straight,ranked
4,35672,0.825033,0.725438,1.0,1.0,1.0,13.449287,"oakland, california",25,m,straight,ranked
5,10444,0.823391,0.722858,1.0,1.0,1.0,19.197039,"berkeley, california",22,m,straight,ranked
6,47397,0.823200,0.722558,1.0,1.0,1.0,23.920071,"san francisco, california",32,m,straight,ranked
7,31015,0.823117,0.723199,1.0,1.0,1.0,23.920071,"san francisco, california",31,m,straight,ranked
8,41489,0.822003,0.720677,1.0,1.0,1.0,31.716076,"belvedere tiburon, california",23,m,straight,ranked
9,17586,0.820568,0.718421,1.0,1.0,1.0,23.920071,"san francisco, california",24,m,straight,ranked


In [142]:
def add_reasons_simple(df_in):
    df = df_in.copy()

    def r(row):
        parts = []
        # text similarity
        if "text_sim" in row and pd.notna(row.text_sim):
            parts.append(f"text~{row.text_sim:.2f}")
        # age rule
        if "age_score" in row and pd.notna(row.age_score):
            parts.append(f"age={int(row.age_score)}")
        # lifestyle
        if "lifestyle_score" in row and pd.notna(row.lifestyle_score):
            parts.append(f"life~{row.lifestyle_score:.2f}")
        # distance
        if "distance_km" in row and pd.notna(row.distance_km):
            parts.append(f"{int(round(row.distance_km))} km")
        # source tag (ranked/suggested)
        if "source" in row and pd.notna(row.source):
            parts.append(str(row.source))
        return " • ".join(parts)

    df["reasons"] = df.apply(r, axis=1)
    return df

In [143]:
tinder_suggestions_df = add_reasons_simple(tinder_suggestions_df)
tinder_suggestions_df[["user_id","tinder_score","reasons"]].head(15)

,user_id,tinder_score,reasons
0,38159,0.836038,text~0.74 • age=1 • life~1.00 • 26 km • ranked
1,3717,0.830340,text~0.73 • age=1 • life~1.00 • 31 km • ranked
2,15046,0.827298,text~0.73 • age=1 • life~1.00 • 24 km • ranked
3,16398,0.825377,text~0.73 • age=1 • life~1.00 • 31 km • ranked
4,35672,0.825033,text~0.73 • age=1 • life~1.00 • 13 km • ranked
5,10444,0.823391,text~0.72 • age=1 • life~1.00 • 19 km • ranked
6,47397,0.823200,text~0.72 • age=1 • life~1.00 • 24 km • ranked
7,31015,0.823117,text~0.72 • age=1 • life~1.00 • 24 km • ranked
8,41489,0.822003,text~0.72 • age=1 • life~1.00 • 32 km • ranked
9,17586,0.820568,text~0.72 • age=1 • life~1.00 • 24 km • ranked


In [168]:
# ensure bio_text is present
if "bio_text" not in tinder_suggestions_df.columns:
    tinder_suggestions = tinder_suggestions_df.merge(df[["user_id", "bio_text"]],on="user_id",how="left")

In [163]:
HOBBY_KEYWORDS = {"hiking","hike","climb","gym","fitness","yoga","run","running","bike","cycling","swim","travel","trip",
                  "roadtrip","explore","music","concert","festival","dj","guitar","piano","food","restaurant","cook","cooking",
                  "bake","baking","coffee","brunch","movie","film","cinema","netflix","series","tv","anime",
                  "art","museum","gallery","paint","painting","draw","photography","dog","dogs","cat","cats","pet","pets",
                  "boardgames","games","gaming","reading","books"}

def _highlight_keywords(text, max_hits=6):
    if not isinstance(text, str) or not text:
        return ""
    lower = text.lower()
    hits = sorted({kw for kw in HOBBY_KEYWORDS if kw in lower})
    return " • ".join(hits[:max_hits])

In [166]:
def add_reasons(df_in, bio_col="bio_text"):
    df = df_in.copy()

    def r(row):
        parts = []
        if "text_sim" in row and pd.notna(row.text_sim) and row.text_sim > 0.70:
            parts.append("You write alike")
        if "age_score" in row and pd.notna(row.age_score) and row.age_score == 1:
            parts.append("Similar age")
        if "lifestyle_score" in row and pd.notna(row.lifestyle_score) and row.lifestyle_score > 0.8:
            parts.append("Lifestyle match")
        if "distance_km" in row and pd.notna(row.distance_km) and row.distance_km < 10:
            parts.append("Nearby")
        if "source" in row and pd.notna(row.source):
            parts.append(f"From {row.source}")
        return " • ".join(parts)

    df["reasons"] = df.apply(r, axis=1)
    if bio_col not in df.columns:
        df[bio_col] = ""
    df["details"] = df[bio_col].apply(_highlight_keywords)   # <- hobbies only
    return df


In [169]:
# Safe: no duplicate columns, works whether or not bio_text already exists
bio_map = df.set_index("user_id")["bio_text"]

# If you want to overwrite any existing bio_text with the master copy:
tinder_suggestions_df["bio_text"] = tinder_suggestions_df["user_id"].map(bio_map)

# Now add reasons/hobbies
tinder_suggestions_df = add_reasons(tinder_suggestions_df, bio_col="bio_text")
tinder_suggestions_df[["user_id","tinder_score","reasons","details"]].head(15)

,user_id,tinder_score,reasons,details
0,38159,0.836038,You write alike • Similar age • Lifestyle matc...,art • cook • cooking • gym • reading • travel
1,3717,0.830340,You write alike • Similar age • Lifestyle matc...,art • books • food • music • pet • pets
2,15046,0.827298,You write alike • Similar age • Lifestyle matc...,coffee • concert • fitness • food • music • tv
3,16398,0.825377,You write alike • Similar age • Lifestyle matc...,art • books • cat • coffee • food • guitar
4,35672,0.825033,You write alike • Similar age • Lifestyle matc...,art • books • cat • concert • dog • dogs
5,10444,0.823391,You write alike • Similar age • Lifestyle matc...,art • bike • books • explore • film • food
6,47397,0.823200,You write alike • Similar age • Lifestyle matc...,art • books • cook • cooking • food • music
7,31015,0.823117,You write alike • Similar age • Lifestyle matc...,art • bike • books • cat • cook • cooking
8,41489,0.822003,You write alike • Similar age • Lifestyle matc...,art • cook • dog • dogs • movie • music
9,17586,0.820568,You write alike • Similar age • Lifestyle matc...,art • books • cat • food • games • movie


In [190]:
def summarize_tinder_elo(state: pd.DataFrame, log: pd.DataFrame, df_master: pd.DataFrame) -> pd.DataFrame:
    """
    Produces: user_id, tinder_score (=elo), reasons, details,
              right_swipe_rate, match_rate, opp_avg_elo, schedule_strength
    (NaN-safe version)
    """
    INIT = INIT_ELO  # alias

    if log.empty:
        out = state[['user_id','elo','games']].copy()
        out['right_swipe_rate'] = 0.0
        out['match_rate'] = 0.0
        out['opp_avg_elo'] = INIT
        out['schedule_strength'] = 0
    else:
        u_view = log.rename(columns={'u':'user_id','v':'opp_id','u_right':'right'})[['user_id','opp_id','right','match']]
        v_view = log.rename(columns={'v':'user_id','u':'opp_id','v_right':'right'})[['user_id','opp_id','right','match']]
        both = pd.concat([u_view, v_view], ignore_index=True)

        opp_elo = state[['user_id','elo']].rename(columns={'user_id':'opp_id','elo':'opp_elo'})
        both = both.merge(opp_elo, on='opp_id', how='left')

        agg = both.groupby('user_id', as_index=False).agg(
            right_swipe_rate=('right','mean'),
            match_rate=('match','mean'),
            opp_avg_elo=('opp_elo','mean'),
            schedule_strength=('right','count')
        )

        out = state.merge(agg, on='user_id', how='left')

    # ---- NaN-safe fills (important when some users had zero encounters) ----
    out['right_swipe_rate'] = out['right_swipe_rate'].fillna(0.0)
    out['match_rate']       = out['match_rate'].fillna(0.0)
    out['opp_avg_elo']      = out['opp_avg_elo'].fillna(INIT)
    out['schedule_strength']= out['schedule_strength'].fillna(0).astype(int)

    # reasons/details
    def mk_reasons(r):
        parts = [f"Elo {r['elo']:.0f} from {int(r.get('games',0))} encounters"]
        if r['match_rate'] >= 0.25: parts.append("high mutual-match rate")
        elif r['match_rate'] <= 0.05 and r['schedule_strength'] > 0: parts.append("low mutual-match rate")
        if r['opp_avg_elo'] > INIT + 40: parts.append("faced strong opponents")
        elif r['opp_avg_elo'] < INIT - 40: parts.append("faced weaker opponents")
        return "; ".join(parts)

    def mk_details(r):
        return (f"right_swipe_rate={r['right_swipe_rate']:.2f}, "
                f"match_rate={r['match_rate']:.2f}, "
                f"opp_avg_elo={r['opp_avg_elo']:.1f}, "
                f"schedule_strength={r['schedule_strength']}")

    out['reasons'] = out.apply(mk_reasons, axis=1)
    out['details'] = out.apply(mk_details, axis=1)

    out = out.rename(columns={'elo':'tinder_score'})
    keep = ['user_id','tinder_score','reasons','details',
            'right_swipe_rate','match_rate','opp_avg_elo','schedule_strength']
    for c in keep:
        if c not in out.columns:
            out[c] = np.nan
    return out[keep].sort_values('tinder_score', ascending=False).reset_index(drop=True)

In [ ]:
"""
How ELO works
Use 4 users: A, B, C, D.
(Think of them as real users from df after we sub‑sample with FAST_USERS.)

0) the knobs (for this toy run)
POOL_K_FOR_PAIRS = 2 → for each user, ask get_matches for up to 2 candidates.
N_ROUNDS = 2, ENCOUNTERS_PER_U = 2 → at most 2 encounters per user per round (≈4 total).
PAIR_CAP is large so we don’t trim.
Everyone starts at Elo 1200.

1) build the candidate graph (directed edges)
Pretend your get_matches returns:
get_matches(A) → {B, C}
get_matches(B) → {A, D}
get_matches(C) → {A, B}
get_matches(D) → {B}

From this, we record directed edges (u → v):
A→B, A→C
B→A, B→D
C→A, C→B
D→B

So the raw edge set is:{ (A,B), (A,C), (B,A), (B,D), (C,A), (C,B), (D,B) }

2) keep only mutual pairs, then make them undirected
Mutual means both directions exist:
(A,B) and (B,A) → mutual
(A,C) and (C,A) → mutual
(B,D) and (D,B) → mutual

Make them undirected + unique (sort endpoints):{ {A,B}, {A,C}, {B,D} }
These are the pairs we’ll sample encounters from during the rounds.

3) set up users (state)
Everyone starts with: elo = 1200
true_attractiveness = random (since there are no images).

For illustration: A: 0.8, B: 0.1, C: 0.5, D: −0.2
This hidden number drives swipe probability via a sigmoid of the difference:
p(i swipes right on j) = sigmoid( B0 + B1 * (a_i - a_j) + noise )

4) Round 1 — sample encounters, simulate swipes, prepare Elo updates
Each user has a quota of ENCOUNTERS_PER_U = 2 this round.
We shuffle the pair list {A–B, A–C, B–D} and walk it once, adding a pair only if both have quota left.

Now simulate swipes:
A vs B:
p(A→B) = sigmoid(2*(0.8−0.1)) ≈ 0.80 → likely right
p(B→A) = sigmoid(2*(0.1−0.8)) ≈ 0.20 → likely left
Outcome (example): A right, B left ⇒ no match

A vs C:
p(A→C) ≈ sigmoid(2*(0.8−0.5)) = sigmoid(0.6) ≈ 0.65 → likely right
p(C→A) ≈ sigmoid(2*(0.5−0.8)) = sigmoid(−0.6) ≈ 0.35 → could go either way
Outcome (example): both right ⇒ mutual match

B vs D:
p(B→D) ≈ sigmoid(2*(0.1−(−0.2))) = sigmoid(0.6) ≈ 0.65 → likely right
p(D→B) ≈ sigmoid(−0.6) ≈ 0.35 → likely left
Outcome (example): B right, D left ⇒ no match

Elo scoring for a single encounter
Rules: 
Right swipe = 0.5
Mutual match adds +0.5 (so mutual gives 1.0 to both)
Left swipe with no mutual = 0.0
The expected score is the Elo expectation vs the opponent:E(i) = 1 / (1 + 10^((elo_j - elo_i) / 400))

With both at 1200, E(i)=0.5.
The Elo update per user is:
Δ = K * (actual − expected)
(K decays slightly as games increases; at the start it’s ~24.)

Apply that to Round 1 examples
A vs B (A right / B left; no match)
A: actual = 0.5, expected ≈ 0.5 → ΔA ≈ 0
B: actual = 0.0, expected ≈ 0.5 → ΔB ≈ −12 (with K≈24)

A vs C (mutual match)
A: actual = 1.0, expected ≈ 0.5 → ΔA ≈ +12
C: actual = 1.0, expected ≈ 0.5 → ΔC ≈ +12

B vs D (B right / D left; no match)
After B’s first drop, B’s Elo is ≈1188 vs D=1200, so E(B) ≈ 0.48:
B: actual = 0.5 → ΔB ≈ +0.4 (small recovery)
D: actual = 0.0, expected ≈ 0.52 → ΔD ≈ −12.5

End of Round 1 (approx):A ≈ 1212, B ≈ 1188.4, C ≈ 1212, D ≈ 1187.5

Everyone’s games increased by the number of encounters they appeared in.

5) Round 2 — repeat sampling & swipes, more Elo updates
Quotas reset to 2 per user; we sample pairs again from {A–B, A–C, B–D} (subject to quotas).
New swipes done, new matches, and another set of Elo deltas.

After 2 rounds:
A: high Elo (did well, got at least one mutual vs decent opponents)
C: also high (mutual with A)
B: around/below 1200 (few/no mutuals, some right swipes unreciprocated)
D: below 1200 (few/no mutuals, faced stronger A/B/C)

6) summarize into your table
For each user we compute:
tinder_score = final Elo.
right_swipe_rate = (# right swipes) / (# encounters).
match_rate = (# mutual matches) / (# encounters).
opp_avg_elo = average Elo of opponents they faced.
schedule_strength = total encounters they appeared in (how much data we have).
reasons = text like "Elo 1234 from 8 encounters; high mutual-match rate; faced strong opponents"
details = compact metrics string (right_swipe_rate=..., match_rate=..., ...)

In [194]:
# ===== ELO Simulation ==========================
from tqdm.auto import tqdm

#configs
FAST_USERS          = 600    #we randomly pick 600 users to simulate; if None, we use all users.
POOL_K_FOR_PAIRS    = 100    #When building candidates for each user, ask get_matches for up to 100 potential partners.
PAIR_CAP            = 80_000 #Hard cap on how many (unique undirected) pairs we’ll keep, to bound work.
N_ROUNDS            = 8      
ENCOUNTERS_PER_U    = 10     
#Simulation runs 8 rounds; each round tries to give each user ~10 encounters (subject to availability).

def build_mutual_pairs_fast(df_users: pd.DataFrame,pool_k: int = POOL_K_FOR_PAIRS,
                            max_km: float = MAX_KM_FOR_PAIRS,pair_cap: int = PAIR_CAP) -> pd.DataFrame:
    """
    Faster mutual-pair builder:
      - optional user sub-sample
      - progress bar
      - caches candidate lists
      - set math for mutuals
      - caps total pairs
    """
    # Ensure int ids
    users_all = df_users[['user_id']].dropna().drop_duplicates().copy()
    users_all['user_id'] = users_all['user_id'].astype(int)

    # Optional sub-sample
    if FAST_USERS is not None and FAST_USERS < len(users_all):
        users = users_all.sample(FAST_USERS, random_state=7)['user_id'].tolist()
    else:
        users = users_all['user_id'].tolist()

    cand_map = {}
    edges = []

    for uid in tqdm(users, desc="Building candidate edges", leave=False):
        c = get_matches(user_id=uid, k=pool_k,use_orientation=True, use_location=True, max_km=max_km,
                    use_age=True, use_diet=False, use_drinks=False, use_smokes=False, use_drugs=False,
                    drinks_strict=False, smokes_strict=False, drugs_strict=False, allow_missing=True)
        cands = c['user_id'].astype(int).tolist() if 'user_id' in c.columns else []

        if cands:
            # extend directed edges u->v
            edges.extend((uid, v) for v in cands if v != uid)

        if pair_cap is not None and len(edges) >= pair_cap:
            break

    # Mutual detection with sets
    dir_set = set(edges)
    rev_set = {(v, u) for (u, v) in dir_set}
    mutual_dir = dir_set & rev_set

    undirected = {(min(u, v), max(u, v)) for (u, v) in mutual_dir}

    if pair_cap is not None and len(undirected) > pair_cap:
        undirected = set(list(undirected)[:pair_cap])

    pairs = pd.DataFrame(list(undirected), columns=['u', 'v'])
    return pairs

def run_tinder_elo_with_progress(df_users: pd.DataFrame,pairs: pd.DataFrame,
                                 rounds: int = N_ROUNDS,encounters_per_user: int = ENCOUNTERS_PER_U):
    """
    Initialize Elo state
    state = init_elo_state(df_users) sets everyone to INIT_ELO and gives each a 
    random true_attractiveness. (Defined in your Elo core cell.)"""
    
    # Ensure int ids and drop dups
    df_users = df_users[['user_id']].dropna().drop_duplicates().copy()
    df_users['user_id'] = df_users['user_id'].astype(int)

    # init fresh state
    state = init_elo_state(df_users)

    logs = []
    for _ in tqdm(range(rounds), desc="Simulating Elo rounds", leave=False):
        state, L = simulate_round(state, pairs, encounters_per_user=encounters_per_user)
        logs.append(L)
    log = pd.concat(logs, ignore_index=True) if logs else pd.DataFrame(columns=['u','v','u_right','v_right','match'])
    return state, log
    """Each call to simulate_round (from the core cell) samples a subset of pairs, 
    simulates two‑sided swipes, updates Elo, and returns a per‑round log L."""

# ------------------- RUN (fast) -------------------
# (a) minimal user frame
df_users_min = df[['user_id']].dropna().drop_duplicates().astype({'user_id':'int64'}).reset_index(drop=True)

# (b) fast mutual pairs with progress bar
pairs = build_mutual_pairs_fast(df_users_min,pool_k=POOL_K_FOR_PAIRS,
                                max_km=MAX_KM_FOR_PAIRS,pair_cap=PAIR_CAP)

# (c) Elo with progress bar
if pairs.empty:
    print("⚠️ No mutual pairs found. Try raising POOL_K_FOR_PAIRS or FAST_USERS.")
else:
    # (c) Elo with progress bar
    elo_state, elo_log = run_tinder_elo_with_progress(df_users_min, pairs,rounds=N_ROUNDS,
        encounters_per_user=ENCOUNTERS_PER_U)

# (d) summarize (use the NaN-safe summarize_tinder_elo we fixed earlier)
tinder_elo = summarize_tinder_elo(elo_state, elo_log, df_master=df)

# (e) show (no prints)
tinder_elo.head(10)
# ============================================================================ 

Building candidate edges:   0%|          | 0/600 [00:00<?, ?it/s]

Start pool: 5000 (max_candidates=5000)
After orientation: 2098
Unfiltered: 2098 | After location filter (≤50 km): 2092
After location (≤50 km): 2092
After age rule: 1735
Start pool: 5000 (max_candidates=5000)
After orientation: 2323
Unfiltered: 2323 | After location filter (≤50 km): 2290
After location (≤50 km): 2290
After age rule: 1495
Start pool: 5000 (max_candidates=5000)
After orientation: 2466
Unfiltered: 2466 | After location filter (≤50 km): 2266
After location (≤50 km): 2266
After age rule: 690
Start pool: 5000 (max_candidates=5000)
After orientation: 370
Unfiltered: 370 | After location filter (≤50 km): 346
After location (≤50 km): 346
After age rule: 222
Start pool: 5000 (max_candidates=5000)
After orientation: 2137
Unfiltered: 2137 | After location filter (≤50 km): 1956
After location (≤50 km): 1956
After age rule: 1082
Start pool: 5000 (max_candidates=5000)
After orientation: 1971
Unfiltered: 1971 | After location filter (≤50 km): 1966
After location (≤50 km): 1966
After a

Simulating Elo rounds:   0%|          | 0/8 [00:00<?, ?it/s]

,user_id,tinder_score,reasons,details,right_swipe_rate,match_rate,opp_avg_elo,schedule_strength
0,17355,1230.840926,Elo 1231 from 8 encounters; high mutual-match ...,"right_swipe_rate=0.88, match_rate=0.50, opp_av...",0.875,0.5000,1203.037346,8
1,330,1222.622693,Elo 1223 from 8 encounters; high mutual-match ...,"right_swipe_rate=0.75, match_rate=0.50, opp_av...",0.750,0.5000,1219.833823,8
2,37277,1219.833823,Elo 1220 from 16 encounters; high mutual-match...,"right_swipe_rate=0.75, match_rate=0.44, opp_av...",0.750,0.4375,1167.441798,16
3,17079,1217.216942,Elo 1217 from 8 encounters; high mutual-match ...,"right_swipe_rate=0.88, match_rate=0.38, opp_av...",0.875,0.3750,1182.052600,8
4,40440,1208.879557,Elo 1209 from 8 encounters; high mutual-match ...,"right_swipe_rate=0.75, match_rate=0.38, opp_av...",0.750,0.3750,1190.356999,8
5,21468,1208.543625,Elo 1209 from 8 encounters; high mutual-match ...,"right_swipe_rate=0.75, match_rate=0.38, opp_av...",0.750,0.3750,1179.320797,8
6,20679,1207.141922,Elo 1207 from 8 encounters; high mutual-match ...,"right_swipe_rate=0.75, match_rate=0.38, opp_av...",0.750,0.3750,1192.049536,8
7,29818,1207.049340,Elo 1207 from 8 encounters; high mutual-match ...,"right_swipe_rate=0.88, match_rate=0.25, opp_av...",0.875,0.2500,1160.230704,8
8,10673,1206.907123,Elo 1207 from 8 encounters; high mutual-match ...,"right_swipe_rate=0.88, match_rate=0.25, opp_av...",0.875,0.2500,1158.787999,8
9,4321,1206.847937,Elo 1207 from 8 encounters; high mutual-match ...,"right_swipe_rate=0.88, match_rate=0.25, opp_av...",0.875,0.2500,1158.262445,8


In [ ]:
"""
Row example (user_id = 55144, tinder_score ≈ 1234)
- user_id: the identifier of that simulated user.
- tinder_score = 1234.19: their final Elo rating. 
    Higher than 1200 means they “performed well” (lots of matches against decent opponents).
-reasons: a text summary: “Elo 1234 from 8 encounters; high mutual-match …” → 
   says they had 8 rounds of encounters and a relatively strong swipe/match pattern.
- details: breakdown: right_swipe_rate=0.50, match_rate=0.50, opp_avg_elo=1212.5 
            →swiped right half the time,
            about half of those turned into mutual matches,
            their opponents on average were rated around 1212 Elo.
- right_swipe_rate = 0.875: 
    out of all encounters, 87.5% swipes were right. (Sometimes differs slightly from the text summary rounding).
- match_rate = 0.5000: 
    50% of their swipes resulted in mutual likes.
- opp_avg_elo = 1212.5: 
     the “strength of schedule” — how strong their average opponents were.
- schedule_strength = 8: 
     how many encounters they actually had (here, exactly 8).
"""